In [ ]:
!pip install pandas
!pip install openpyxl
!pip install joblib
!pip install requests
!pip install geojson
!pip install sklearn
!pip install numpy

In [ ]:
import pandas as pd
import unicodedata
import requests
import geojson
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# STAREZ data

starez_excel_file = '/content/Starez.xlsx'

# Load the Excel file
df_starez_data = pd.read_excel(starez_excel_file, sheet_name='Data', engine='openpyxl', header=0)
df_starez_ciselnik = pd.read_excel(starez_excel_file, sheet_name='Číselník', engine='openpyxl', header=None, skiprows=1)

In [ ]:
df_starez_data.rename(columns={df_starez_data.columns[0]: 'Zkratka'}, inplace=True)
df_starez_data.head()

In [ ]:
# odstraneni duplikatu
# Najdi index řádku, kde je ve sloupci 'Zkratka' hodnota 'Celkový součet'
#index_to_drop_from = df_starez_data[df_starez_data['Zkratka'] == 'Celkový součet'].index[0]

# Dropni všechny řádky od tohoto indexu (včetně) dolů a přepiš původní DataFrame
#df_starez_data = df_starez_data[:index_to_drop_from]

# Zobraz prvních pár řádků vyčištěného DataFrame
#df_starez_data.head()

In [ ]:
df_starez_data.shape[0]

In [ ]:
df_starez_data = df_starez_data[df_starez_data['Zkratka'] != 'ESHOP']

In [ ]:
df_starez_data.shape[0]

In [ ]:
df_starez_ciselnik.columns = ['Identifikator', 'Zkratka', 'Název sportoviště']
df_starez_ciselnik

In [ ]:
# Odstraníme řádek, kde je ve sloupci 'Zkratka' hodnota 'ESHOP'
df_starez_ciselnik = df_starez_ciselnik[df_starez_ciselnik['Zkratka'] != 'ESHOP']

# Zobrazíme výsledný DataFrame
df_starez_ciselnik

In [ ]:
# Proveď sloučení na základě sloupce 'Zkratka'
df_starez_merged = pd.merge(df_starez_data, df_starez_ciselnik[['Zkratka', 'Název sportoviště']], on='Zkratka', how='left')

# Nahradíš první sloupec v Data novým názvem sportoviště (třetí sloupec z Číselníku)
df_starez_merged['Zkratka'] = df_starez_merged['Název sportoviště']

# Odstraníš pomocný sloupec 'Název sportoviště'
df_starez_merged.drop(columns=['Název sportoviště'], inplace=True)

# Prejmenovani mesice
df_starez_merged.rename(columns={'Měsíc ': 'Měsíc'}, inplace=True)

# Zobraz první řádky výsledku
df_starez_merged.head()

In [ ]:
# WEATHER DATA HISTORY

# Load the "Data" sheet from the Excel file
#df_weather_historical = pd.read_excel('/content/B2BTUR01.xlsx', sheet_name='teplota průměrná', engine='openpyxl', header=3)
df_weather_historical_dict = pd.read_excel('/content/B2BTUR01.xlsx', sheet_name=None, engine='openpyxl', header=3)


# Function to remove diacritics from a string
def remove_diacritics(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Rename the specific sheet in the DataFrame
if 'rychlost větru ' in df_weather_historical_dict:
    df_weather_historical_dict['rychlost větru'] = df_weather_historical_dict.pop('rychlost větru ')

# List of sheets to merge
sheets_to_merge = [
    'teplota průměrná', 'teplota maximální', 'teplota minimální',
    'rychlost větru', 'tlak vzduchu', 'vlhkost vzduchu',
    'úhrn srážek', 'celková výška sněhu', 'sluneční svit'
]

# List to store melted DataFrames
melted_dfs = []

# Process each sheet
for sheet_name in sheets_to_merge:
    df = df_weather_historical_dict[sheet_name]
    # Melt the DataFrame
    melted_df = df.melt(id_vars=['rok', 'měsíc'],
                        value_vars=[str(i) + '.' for i in range(1, 32)],  # Add '.' to match the columns
                        var_name='day', value_name='value')

    # Rename the 'value' column to include the sheet name and remove diacritics
    melted_df.rename(columns={'value': f'{remove_diacritics(sheet_name)}_value'}, inplace=True)

    # Convert 'day' to numeric for merging purposes
    melted_df['day'] = pd.to_numeric(melted_df['day'])

    # Add the melted DataFrame to the list
    melted_dfs.append(melted_df)

# Merge all melted DataFrames
merged_df = melted_dfs[0]
for df in melted_dfs[1:]:
    merged_df = pd.merge(merged_df, df, on=['rok', 'měsíc', 'day'], how='outer')

# Change type for day
merged_df['day'] = merged_df['day'].astype('Int64')

# Display the final merged DataFrame
merged_df.head()


# Display the first few rows
#df_weather_historical.head()

In [ ]:
merged_df.rename(columns={'rok': 'Rok', 'měsíc': 'Měsíc', 'day': 'Den'}, inplace=True)
merged_df.head()

In [ ]:
df_starez_merged[['Rok', 'Měsíc', 'Den', 'Veřejnost', 'Hodina', 'Abonenti', 'Celkem']] = df_starez_merged[['Rok', 'Měsíc', 'Den', 'Veřejnost', 'Hodina', 'Abonenti', 'Celkem']].astype("Int64")

# Provedeme sloučení na základě Rok, Měsíc a Den
df_train = pd.merge(df_starez_merged, merged_df, on=['Rok', 'Měsíc', 'Den'], how='left')

# Identify columns that end with 'value'
value_columns = df_train.filter(regex='value$').columns

# Round those columns to 1 decimal place
df_train[value_columns] = df_train[value_columns].round(1)


# Display the merged dataframe
df_train.head()

# DATA PREPROCESSING

In [ ]:
df_train.info()

In [ ]:
df_train = df_train[df_train['Rok'] <= 2022]
df_train.head()

In [ ]:
df_train[['Veřejnost', 'Abonenti']] = df_train[['Veřejnost', 'Abonenti']].fillna(0).astype('Int64')
df_train.head()

In [ ]:
df_train['Celkem'] = df_train['Veřejnost'] + df_train['Abonenti']
df_train.head()

In [ ]:
# Count of missing values in each column
missing_values = df_train.isnull().sum()
print(missing_values[missing_values > 0])  # Display only columns with missing values

In [ ]:
# Spočítáme průměrnou hodnotu sloupce 'slunecni svit', přičemž ignorujeme NaN hodnoty
mean_value = df_train['slunecni svit_value'].mean()

# Nahradíme chybějící hodnoty (NaN) průměrnou hodnotou
df_train['slunecni svit_value'] = df_train['slunecni svit_value'].fillna(mean_value)

# Zobrazíme první řádky, abychom ověřili výsledek
df_train.head()

In [ ]:
# Count of missing values in each column
missing_values = df_train.isnull().sum()
print(missing_values[missing_values > 0])  # Display only columns with missing values

In [ ]:
df_train.info()

In [ ]:
# Odstraníme řádky, které mají NaN ve sloupci 'Zkratka'
df_train = df_train.dropna(subset=['Zkratka'])

mean_value = df_train['rychlost vetru_value'].mean()
# Use .loc to avoid the SettingWithCopyWarning
df_train.loc[:, 'rychlost vetru_value'] = df_train['rychlost vetru_value'].fillna(mean_value)

In [ ]:
df_train['day_of_week'] = pd.to_datetime(df_train[['Rok', 'Měsíc', 'Den']].rename(columns={'Rok': 'year', 'Měsíc': 'month', 'Den': 'day'})).dt.dayofweek
# Assuming df is your DataFrame
df_train['mean_abonenti'] = df_train.groupby(['Měsíc', 'Den'])['Abonenti'].transform('mean')
df_train['mean_verejnost'] = df_train.groupby(['Měsíc', 'Den'])['Veřejnost'].transform('mean')

In [ ]:
# Step 1: Calculate mean values for the historical features
mean_historical_values = df_train.groupby(['Měsíc', 'Den', 'Hodina'])[
    ['rychlost vetru_value',
     'tlak vzduchu_value',
     'vlhkost vzduchu_value',
     'uhrn srazek_value']].transform('mean')

# Step 2: Add the mean values as new columns to df_train
df_train[['mean_rychlost_vetru',
           'mean_tlak_vzduchu',
           'mean_vlhkost_vzduchu',
           'mean_uhrn_srazek']] = mean_historical_values

# Step 3: Ensure your date and hour columns are in the correct format
df_train['Date'] = pd.to_datetime(df_train[['Rok', 'Měsíc', 'Den']].rename(columns={'Rok': 'year', 'Měsíc': 'month', 'Den': 'day'}))

# Step 4: Group by Date and Hour, and calculate mean for the specified columns
mean_values = df_train.groupby(['Date', 'Hodina'])[['teplota maximalni_value',
                                                      'teplota minimalni_value',
                                                      'Abonenti',
                                                      'Veřejnost',
                                                      'celkova vyska snehu_value',
                                                      'slunecni svit_value',
                                                      'mean_rychlost_vetru',
                                                      'mean_tlak_vzduchu',
                                                      'mean_vlhkost_vzduchu',
                                                      'mean_uhrn_srazek']].mean().reset_index()

# Rename columns for clarity (optional)
mean_values.columns = ['Date', 'Hodina', 'Mean_Teplota_Max', 'Mean_Teplota_Min',
                       'Mean_Abonenti', 'Mean_Verejnost',
                       'Mean_Celkova_Vyska_Snehu', 'Mean_Slunecni_Svit',
                       'Mean_Rychlost_Vetru', 'Mean_Tlak_Vzduchu',
                       'Mean_Vlhkost_Vzduchu', 'Mean_Uhrn_Srazek']

# Step 5: Save the result to a CSV file
mean_values.to_csv('mean_values_hourly.csv', index=False)

# Display the first few rows of the mean values DataFrame
mean_values.head()

In [ ]:
# 'rychlost vetru_value', 'tlak vzduchu_value', 'vlhkost vzduchu_value', 'uhrn srazek_value'
# Calculate mean values for the historical features

# DROPPING VALUES
# Remove not needed features
df_train = df_train.drop(columns=['Rok', 'Abonenti', 'Veřejnost',
                                  'teplota maximalni_value', 'teplota minimalni_value', 'celkova vyska snehu_value', 'slunecni svit_value',
                                  'rychlost vetru_value', 'tlak vzduchu_value', 'vlhkost vzduchu_value', 'uhrn srazek_value',
                                  'mean_vlhkost_vzduchu'])

# Zobrazíme první řádky výsledného DataFrame
df_train.head()

In [ ]:
# Count of missing values in each column
missing_values = df_train.isnull().sum()
print(missing_values[missing_values > 0])  # Display only columns with missing values

In [ ]:
import joblib

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Assuming you have a DataFrame `df` with your data

# Encode categorical variables (e.g., 'Zkratka')
df_encoded = pd.get_dummies(df_train, columns=['Zkratka'], drop_first=True)
# Assuming df_encoded is your DataFrame after one-hot encoding
#print("New columns after one-hot encoding:")
#print(df_encoded.columns.tolist())

# Define features and target variable
X = df_encoded.drop(columns=['Celkem'])  # All columns except the target
y = df_encoded['Celkem']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
#model = RandomForestRegressor(n_estimators=100, random_state=42)
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

# Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print("Mean Absolute Percentage Error (MAPE):", mape)

# R-squared (R²)
r2 = r2_score(y_test, y_pred)
print("R-squared (R²):", r2)

In [ ]:
# Save the trained model
joblib.dump(model, 'random_forest_model.pkl')

# Load the model
loaded_model = joblib.load('random_forest_model.pkl')

# Make predictions with the loaded model
predictions = loaded_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the size of the plot
plt.figure(figsize=(12, 8))

# Create a heatmap

# Select only numeric columns from df_train
numeric_df = df_train.select_dtypes(include=['float64', 'int64'])

# Calculate the correlation matrix
correlation_matrix = numeric_df.corr()

sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar=True)

# Show the plot
plt.title('Correlation Matrix')
plt.show()
